In [12]:
import string
import numpy as np
import pandas as pd
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from glob import glob

import cv2
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from tensorflow.keras.applications.resnet import ResNet101
import csv
from rouge import Rouge

### For Google Colab

In [3]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
home = './'

In [5]:
# Loading a text file into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# Loading a binary file into memory
def load_doc_bin(filename):
    file = open(filename, 'rb')
    text = load(file)
    file.close()
    return text


### Mapping image with descriptions

In [38]:
# Mapping image with descriptions
def all_img_captions(filename):
    file = load_doc(filename)
    captions = file.split('\n')

    descriptions ={}
    for caption in captions[1:]:
        cap=caption.split(":")
        img = cap[0].strip()
        caption_ = cap[1].strip()
        
        if img not in descriptions:
            descriptions[img] = [caption_]
        else:
            descriptions[img].append(caption_)
    # print(descriptions)
    return descriptions

### Cleaning text in captions dictionary

In [39]:
#Changing to lower cases, removing punctuations and stop words and also numbers
def cleaning_text(captions):
    table = str.maketrans('','', string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):

            img_caption.replace("-"," ")
            desc = img_caption.split()

            #convert to lower case
            desc = [word.lower() for word in desc]
            #remove punctuation
            desc = [word.translate(table) for word in desc]
            #remove 's and a 
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers
            desc = [word for word in desc if(word.isalpha())]

            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
            
    # print(list(captions.values())[0])
    return captions

### Creating Vocabulary from descriptions

In [40]:
def text_vocabulary(descriptions):
    vocab = set()
    
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    
    # print(vocab)
    return vocab

### Saving cleaned description to a file

In [41]:
#All descriptions in one file 
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()

In [42]:
filename = home + "captions_8k.txt"

descriptions = all_img_captions(filename)
# print(description)/
print("Length of descriptions =" ,len(descriptions))

clean_descriptions = cleaning_text(descriptions)

vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))

save_descriptions(clean_descriptions, home + "descriptions.txt")

Length of descriptions = 8091
Length of vocabulary =  8762


## RESNET MODEL

In [43]:
resnet = ResNet101(
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
# resnet.summary()

### Creating features of image using RESNET

In [44]:
def extract_features(directory):
        features = {}
        img_dir = glob(directory + '\*.jpg')
        # print(len(img_dir))
        for image in img_dir:
            filename = image
            
            image = Image.open(filename)
            image = image.resize((224,224))
            image = np.expand_dims(image, axis=0)
            # print(image.shape)

            filename = filename.split('\\')[1]
            
            # print(image.shape)
            feature = resnet.predict(image)
            # print(feature.shape)
            features[filename] = feature
        return features

### Dumping feature vectors

In [15]:
# features = extract_features('./train_dir')
# dump(features, open( home + "image_features.p","wb"))

In [21]:
# features = load(open("features_10k.p","rb"))
# len(features)

### Loading everything into memory

In [45]:
def load_clean_descriptions(filename):    
    #loading clean_descriptions 
    file = load_doc(filename) 
    descriptions = {} 
    for line in file.split("\n"):         
        words = line.split() 
        if len(words)<1 : 
            continue      
        image, image_caption = words[0], words[1:]          
        if image not in train_features.keys(): 
            continue  
        if image not in descriptions: 
            descriptions[image] = [] 
        desc = '<start> ' + " ".join(image_caption) + ' <end>' 
        descriptions[image].append(desc) 
    return descriptions 
 
def load_features(): 
    #loading all features 
    all_features = load_doc_bin(home + "train_8K_6500.p")
    return all_features


In [46]:
train_features = load_features() 
features = train_features 
train_descriptions = load_clean_descriptions(home + "descriptions.txt") 

print(len(features)) 

6500


### Tokenizer to convert characters to integers

In [47]:
#converting dictionary to list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

from keras.preprocessing.text import Tokenizer

def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer


In [48]:
tokenizer = create_tokenizer(train_descriptions)
# print(tokenizer.word_index)

with open(home + 'tokenizer.p', 'wb') as token_file:
    dump(tokenizer, token_file)

In [49]:
tokenizer = load_doc_bin(home + 'tokenizer.p')
vocab_size = len(tokenizer.word_index) + 1
vocab_size 

7945

### Calculating Max length of descriptions

In [50]:
def max_length_func(descriptions):
    desc_list = dict_to_list(descriptions)
    max_ = 0
    for d in desc_list:
        if len(d.split()) > max_:
            max_ = len(d.split())
            # print(max_, d)
    return max_

max_length = max_length_func(descriptions)
max_length

32

#### Generator for training in-memory and preparing inputs for model

In [51]:
def data_generator(descriptions, features, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            if key not in features.keys():
              continue

            feature = features[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield [[input_image, input_sequence], output_word]

def create_sequences(tokenizer, max_length, desc_list, feature):
    X1, X2, y = list(), list(), list()

    for desc in desc_list:
        # encoding the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):

            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)


[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape

((32, 2048), (32, 32), (32, 7945))

## Resnet and LSTM MODEL

In [52]:
from keras.utils.vis_utils import plot_model

def define_model(vocab_size, max_length):
    
    # features from the CNN model
    cnn_input = Input(shape=(2048,))
    layer = Dropout(0.5)(cnn_input)
    cnn_input = Dense(256, activation='relu')(layer)

    # LSTM sequence model
    lstm_input = Input(shape=(max_length,))
    layer = Embedding(vocab_size, 256, mask_zero=True)(lstm_input)
    layer = Dropout(0.5)(layer)
    lstm_input = LSTM(256)(layer)

    # Merging both models
    merged_model = add([cnn_input, lstm_input])
    merged_model = Dense(256, activation='relu')(merged_model)
    outputs = Dense(vocab_size, activation='softmax')(merged_model)
    
    model = Model(inputs=[cnn_input, lstm_input], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    
    # summary of the model
    # print(model.summary())
    plot_model(model, to_file= home + 'model.png', show_shapes=True)
    
    return model

### Training the model

In [53]:
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)

from tensorflow import keras

#Loading the saved model
model = keras.models.load_model("./model_20.h5")

# epochs = 20
# steps = len(train_descriptions)
# # # os.mkdir(home+"models test")
# for i in range(12, epochs):
#     generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
#     model.fit(generator, epochs=1, steps_per_epoch= steps, verbose=1)
#     model.save(home+"models test1/model_" + str(i) + ".h5")
#     print(model)

Descriptions: train= 6500
Photos: train= 6500
Vocabulary Size: 7945
Description Length:  32


In [54]:
def word_for_id(integer, tokenizer):
 for word, index in tokenizer.word_index.items():
     if index == integer:
         return word
 return None

### Predict the caption

In [55]:
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [56]:
def generate_feature(filename):
    image = Image.open(filename)
    image = image.resize((224,224))
    image = np.expand_dims(image, axis=0)
    return resnet.predict(image)

### Generating captions for all images and storing in a CSV file

In [58]:
f = open(home+"predicted.csv", 'w')
writer = csv.writer(f)
writer.writerow(['Image','Caption Generated'])

test_dir=glob(home+'mod_testdata/*.jpg')
# print(test_dir)
for image in test_dir:
  description = generate_desc(model, tokenizer, generate_feature(image), max_length)
  writer.writerow([image,description])
f.close()
# print(description)

#### Example

In [60]:
filename = "./mod_testdata/3082934678_58534e9d2c.jpg"
description = generate_desc(model, tokenizer, generate_feature(filename), max_length)
print(description)

start dog is running through the grass end


## Evaluating Metrics

#### BLEU Score

In [61]:
from nltk.translate.bleu_score import sentence_bleu

BLEUscore=[]
score=[]
data=pd.read_csv(home+"predicted.csv")
predicted=[]
predict_dict={}

with open(home+'predicted.csv') as file:
    content = file.readlines()
print(content[3], len(content))
for x in content[2::2]:
    y=x.split(',')
    
    predict_dict[y[0].split('\\')[1]]=y[1].split()
# print(predict_dict)

img_dir=glob(home+'mod_testdata/*.jpg')
bleu = 0
for img in img_dir:
    filename=img.split('\\')[1]    
    
    if '(1)' in filename or '(2)' in filename:
        continue

    hypothesis = predict_dict[filename][1:len(predict_dict[filename])-1]
    # print(hypothesis,descriptions[img.split('/')[5]])
    x = descriptions[img.split('\\')[1]]

    act = list()
    for string in x:
        act.append(string.split())

    bleu += sentence_bleu(act, hypothesis)
    
# BLEUscore.append(score)
print("BLUE SCORE:",bleu)


 512


c:\users\436pr\appdata\local\programs\python\python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\users\436pr\appdata\local\programs\python\python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\users\436pr\appdata\local\programs\python\python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower o

KeyError: '1346051107_9cdc14e070.jpg'

#### Meteor Score

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
from nltk.translate.meteor_score import meteor_score
from nltk.translate import meteor

meteor=0
for img in img_dir:
    filename=img.split('\\')[1]
    if '(1)' in filename or '(2)' in filename:
        continue
    hypothesis = predict_dict[filename][1:len(predict_dict[filename])-1]
    x = descriptions[img.split('\\')[1]]

    act = list()
    for string in x:
        act.append(string.split())

    meteor += meteor_score(act, hypothesis)
    
print("Meteor SCORE:",meteor/1500)

### Rouge score

In [ ]:
from nltk.translate.meteor_score import meteor_score
from nltk.translate import meteor

BLEUscore=[]
score=[]
data=pd.read_csv(home+"predicted.csv")
predicted=[]
predict_dict={}

with open(home+'predicted.csv') as file:
    content = file.readlines()

for x in content[2::2]:
    y=x.split(',')
    predict_dict[y[0].split('\\')[1]]=y[1]

img_dir=glob(home+'mod_testdata/*.jpg')
rouge= Rouge()
meteor=0
rouge_1={'r':0.0,'f':0.0,'p':0.0}
rouge_2={'r':0.0,'f':0.0,'p':0.0}
rouge_l={'r':0.0,'f':0.0,'p':0.0}
r1=r2=r3=0
p1=p2=p3=0
f1=f2=f3=0

for img in img_dir:    
    filename=img.split('\\')[1]
    if '(1)' in filename or '(2)' in filename:
    continue
    
    hypothesis = predict_dict[filename][:len(predict_dict[filename])-1]
    # print(hypothesis,descriptions[img.split('/')[5]])
    x = descriptions[img.split('\\')[1]]
    #   print(x)
    #   print(hypothesis)
    act = list()
    act.append(hypothesis)
    for string in hypothesis:
        act.append(string.split())
 
  if len(x)==5:
      for i in range(5):
          r1 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-1']['r']
          p1 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-1']['p']
          f1 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-1']['f']
          r2 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-2']['r']
          p2 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-2']['p']
          f2 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-2']['f']
          r3 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-l']['r']
          p3 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-l']['p']
          f3 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-l']['f']
  else:
      for i in range(4):
          r1 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-1']['r']
          p1 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-1']['p']
          f1 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-1']['f']
          r2 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-2']['r']
          p2 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-2']['p']
          f2 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-2']['f']
          r3 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-l']['r']
          p3 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-l']['p']
          f3 +=rouge.get_scores(hypothesis, x[i])[0]['rouge-l']['f']

print(r1/2000,p1/2000,f1/2000)
print(r2/2000,p2/2000,f2/2000)
print(r3/2000,p3/2000,f3/2000)
#print(r)